In [1]:
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re
import pandas as pd

In [2]:
def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    
    return attributes, cleaned_input
def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        files = os.listdir('./entities/')
        #print(files,'----')
        entities = {}
        for fil in files:
            lines = open('./entities/'+fil).readlines()
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines)
        #print(entities,'----')

        #Extract entity and update it in attributes dict
        for entity in entities:
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
        #print(attributes,'----')
        for entity in entities:
            uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)
        #print(uinput,'----')

        return attributes, uinput

In [3]:
def loadIntent(path, intent):
    print(intent)
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    
    #Scoring Algorithm, can be changed.
    scores = ngrammatch(clean_input)
    
    #choosing here the intent with the highest score
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    print(scores)
    # print clean_input
    #print 'scores', scores
    

    if(current_intent==None):
        if(clean_input=="Travel"):
            return loadIntent('params/newparams.cfg', 'Cab')
        if(clean_input=='Dining'):
            return loadIntent('params/newparams.cfg','pubs')
        else:
            return loadIntent('params/newparams.cfg',scores[-1][0])
    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent

In [4]:
#current_intent=intentIdentifier('order $title',FirstGreeting(),None)
def check_actions(current_intent, attributes, context):
    print(current_intent)
    
    if current_intent.name == 'Cab':
        df=pd.read_csv("cabdata.txt")
    
        x=df.loc[ (df['numofbags'] == attributes['numofbags'] ) &(df['type']==attributes['type'] )&(df['When'] == attributes['When'] )]
        print(x)
        x=x.to_dict(orient='list')
        #+"Number " + 'with '+str(x['numofseats'][0])+'Seats' +'and '+str(x['numofbags'][0])+'lugage space will be arriving '+x['When'][0]
        try:
            print('Cab number: '+ x['CarNum'][0] + ' with '+' Seats: ' +str(x['numofseats'][0]) +' and '+str(x['numofbags'][0])+' luggage space will be arriving '+x['When'][0])
        except:
            print("No car")
    elif current_intent.name=='pubs':
        df=pd.read_csv("pubdata.txt")
        
        x=df.loc[ (df['status'] == attributes['status'] ) &(df['cuisine']==attributes['cuisine'] )]
        print(x)
        x=x.to_dict(orient='list')
        try : 
            print('A table for '+str(x['guests'][0]) +' is booked in '+ x['zone'][0]+' zone. Cuisine: ' + x['cuisine'][0] + ' .Budget choice: '+ x['status'][0] +'.' )
        except:
            print('No pub available with given condition,please try again later.')
    context = IntentComplete()
    return 'action: Task Complete', context

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                #Example of where the context is born, implemented in Contexts.py
                #if para.name=='RegNo':
                    #context = GetRegNo()
                #returning a random prompt frmo available choices.
                return random.choice(para.prompts), context

    return None, context





In [ ]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow, see Contexts.py        
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        #print(self.context,self.current_intent,self.attributes)
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        #print(self.context,self.current_intent,self.attributes)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        #print(self.current_intent)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [ ]:
session = Session()

print ('BOT: Hi! How may I assist you?')

while True:
    inp = input('User: ')
    if inp=='end':
        break
    elif inp=='restart':
        del session
        session=Session()
        print ('BOT: Hi! How may I assist you?')
    else:
        print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you?
User: find me a uber
[('pubs', 0.24436109027256814), ('Cab', 0.32261904761904764)]
Cab
BOT: Where do you want to go?
User: koti
[('Cab', 0.05833333333333333), ('pubs', 0.061090272568142034)]
BOT: For how many people?
User: 5
[('Cab', 0.05833333333333333), ('pubs', 0.061090272568142034)]
BOT: number of bags
User: seven
[('Cab', 0.05833333333333333), ('pubs', 0.061090272568142034)]
BOT: Car type?
User: sedan
[('Cab', 0.05833333333333333), ('pubs', 0.061090272568142034)]
BOT: When do you want it
User: now
[('Cab', 0.05833333333333333), ('pubs', 0.061090272568142034)]
             CarNum  numofseats numofbags   type When
3     TS 95 XJ 3873           5     seven  Sedan  Now
8     TS 72 SV 9483           3     seven  Sedan  Now
29   TS 17 NK 61104           6     seven  Sedan  Now
46    TS 11 KB 7392           7     seven  Sedan  Now
103   TS 72 AK 8592           3     seven  Sedan  Now
227  TS 33 CP 93810           6     seven  Sedan  Now
299   TS 54 YD 5656 